# Градиентный спуск

Сегодняшний герой дня - градиентный спуск - краеугольный камень машинного обучения. Градиент - это направление, в котором функция возрастает (антиградиент - в котором убывает). Это направление определяется производной функции (или частными производными для функций с несколькими переменными). Допустим нам надо подобрать значение некоторых параметров, при которых целевая функция будет минимальна. Для этого мы:

1. Случайным образом инициируем искомые параметры.
2. Рассчитаем значение целевой функции для этих параметров.
3. Посчитаем значение градиента для этиx параметров.
4. Изменим значения искомых параметров на значение градиента с определенным шагом. Т.е. вычтем или прибавим к искомым параметрам (полученным в пункте 1) значение градиента умноженного на какое-нибудь число a (например, 0,01, 0,1, 1 и т.д.). Так же это называется "сделать градиентный шаг". Сам параметр a подбирается из диапазона значений: на каждом шаге выбирается такой параметр, при котором целевая функция минимальна.
5. Пересчитаем значение целевой функции, используя параметры полученные после градиентного шага.
6. Проверяем, не сошелся ли градиентный спуск. Для этого можно использовать несколько способов. Можно сравнить новое целевое значение (пункт 5) со значение из пункта 2. Если оно изменилось менее чем определенную заданную точность (tolerance), то мы считаем, что схождение произошло. А можно так же измерять разницу евклидового расстояния между векторами параметров и тоже сравнивать ее с tolerance.
7. Если имеем схождение, то искомые параметры найдены. Если нет - то запоминаем значения целевой функции и новых параметров, а затем повторяем действия с 3 по 6 пункт, пока градиент не сойдется.

Как видно - это итеративный алгоритм, на практике количество итераций часто ограничивают. Давайте перейдем к практике.


Будем решать задачу линейной регрессии. Линейная регрессия это восстановление зависимости одной переменной (зависимой) от другой (не зависимой). Формула следующая:

$$ y = \beta*x + \alpha + \epsilon$$

где y - зависимая переменная, x - независимая, e - ошибка (надеемся, что очень маленькая), которая зависит от факторов, неучтенных в данной простой модели.

Альфа и бетта же - как раз те параметры, которые мы будем пытаться найти. Линейную регрессию можно восстановить с помощью метода наименьших квадратов (МНК). МНК - это метод, при котором минимизируется сумма квадратов ошибок между реальными значениями y и предсказанными значениями.

$$ ошибка = y_{актуальный} - y_{предсказанный} $$

В квадрат ошибка возводится для того, чтобы при суммировании положительные и отрицательные ошибки не выгашивали друг друга. Давайте рассчитаем производную для этой функции. Т.к. функция зависит от альфа и бетта, то градиент будет состоять из двух частных производных. А что если представить, что альфа тоже домножается на x, но этот x всегда равен единице? Уравнение регрессии от этого не измениться, а вот производную можно будет брать только по бета. Найдем же эту производную. Для начала разложим функцию ошибок в квадрате:

ошибка^2 = (y - b*x - a)^2 = ((y-a)-b*x)^2 = (y-a)^2-2(y-a)*bx+(bx)^2

Теперь можно продифференцировать по бетта (y,x и альфа - константы):

(ошибка^2)' = -2x*(y-a) + 2bx = -2x*(y-a-b)

Множитель в скобках это ни что иное, как функция ошибки, значит производную можно записать так:

-2x*ошибка(x,y,бетта)

Теперь облачим все это в форму кода. Чтобы сделать это с нуля, нам потребуется набор функций для работы с векторами.


In [1]:
from __future__ import division
def dot(v, w):
    """перемножает и суммирует два вектора"""
    return sum(v_i * w_i for v_i, w_i in zip(v, w))
def vector_add(v, w):
    """складывает два вектора по-элементно"""
    return [v_i + w_i for v_i, w_i in zip(v,w)]
def vector_subtract(v, w):
    """вычитает два вектора по-элементно"""
    return [v_i - w_i for v_i, w_i in zip(v,w)]
def scalar_multiply(c, v):
    """умножает каждый элемент вектора на число"""
    return [c * v_i for v_i in v]

Для начала напишем функцию предсказания значения.

In [2]:
def predict(x_i, beta):
    return dot(x_i, beta)

Но где же альфа? Как я написал выше, мы будем считать, что альфа тоже домножается на признак, который всегда равен единице. Для этого к исходным данным нам нужно будет добавлять столбик из единиц. Данная функция принимает на входе вектора x и бетта, перемножает и суммирует их.

Функция ошибки будет выглядеть так:

In [3]:
def error(x_i, y_i, beta):
    return y_i - predict(x_i, beta)

Функция ошибки в квадрате:

In [4]:
def squared_error_i(x_i, y_i, beta):
    return error(x_i, y_i, beta) ** 2

А вот и функция для градиента:

In [5]:
def squared_error_gradient_i(x_i, y_i, beta):
    return [-2 * x_ij * error(x_i, y_i, beta)
            for x_ij in x_i]

Все эти функции написаны для i-ого элемента данных. Надо записать их так же для всех данных сразу.

In [6]:
def squared_error(x,y,beta):
    return sum([squared_error_i(x_i,y_i,beta) 
                for x_i, y_i in zip(x,y)])

def squared_error_gradient(x,y,beta):
    return reduce(vector_add,
                  np.array([squared_error_gradient_i(x_i,y_i,beta)
               for x_i,y_i in zip(x,y)]))

Давайте теперь сгенерируем данные, на которых будем проводить испытания.

In [7]:
import numpy as np
from sklearn.datasets.samples_generator import make_regression
x, y = make_regression(n_samples=100, n_features=1, n_informative=1, 
                        random_state=0, noise=35)
# сразу же вставим колонку с единицами
m, n = np.shape(x)
x = np.c_[np.ones(m), x]
x, y = x.tolist(), y.tolist()

Сегодня я буду реализовать версию пакетного (batch) градиетного спуска, т.е. того, который считает шаг на всех данных сразу. Поэтому значения x и y можно сразуже "зашить" в функцию ошибки и градиетна. Для этого используем функцию partial из functools.

In [8]:
from functools import partial
target_fn = partial(squared_error,x,y)
grad_fn = partial(squared_error_gradient,x,y)

Можно писать сам градиентный спуск.

In [9]:
def gradient_descent(target_fn,grad_fn,beta_0,num_iter,tolerance=0.000001):
    
    step_sizes = [100, 10, 1, 0.1, 0.01, 0.001, 0.0001, 0.00001]
    
    beta = beta_0
    value = target_fn(beta)
    for i in range(num_iter):
        grad = grad_fn(beta)
        next_betas = [vector_subtract(beta,
                                      scalar_multiply(step,grad)) 
                      for step in step_sizes] # делаем шаг
        next_beta = min(next_betas, key=target_fn)
        next_value = target_fn(next_beta)
        if abs(value - next_value) < tolerance:
            return beta
        else:
            beta, value = next_beta, next_value 
    return beta

In [10]:
gradient_descent(target_fn,grad_fn,[0,0],10000)

[-2.8495390955931437, 43.204126172220924]

Можно сделать этот код гораздо красивее без всяких вспомогательных функций. Для этого воспользуемся библиотекой numpy. Переведем x и y обратно в массивы numpy.

In [11]:
x, y = np.array(x), np.array(y)

Потребуются так же функции ошибки, ошибки в квадрате и градиента квадратной ошибки.

In [12]:
def error_num(x,y,beta):
    prediction = np.dot(x, beta)
    return y - prediction

In [13]:
def square_error_num(x,y,beta):
    return np.sum(error_num(x,y,beta)**2)

In [14]:
def square_error_grad_num(x,y,beta):
    return -2*np.dot(x.transpose(), error_num(x,y,beta))

Сделаем из них парциалы.

In [15]:
target_fn_num = partial(square_error_num,x,y)
grad_fn_num = partial(square_error_grad_num,x,y)

Сама функция практически не изменится, за исключением строчки с расчетами шагов - теперь не надо использовать функции substract и scalar_multiply.

In [16]:
def gradient_descent_num(target_fn, grad_fn, steps, num_iter=10000,tolerance=0.000001):
    beta = np.ones(2) # инициируем бетту
    value = target_fn(beta)
    for i in range(num_iter):
        grad = grad_fn(beta)
        # упростилась строчка с градиентным шагом
        next_betas = [(beta-grad*step) for step in steps]
        next_beta = min(next_betas, key=target_fn)
        next_value = target_fn(next_beta)
        if abs(value - next_value) < tolerance:
            return beta
        else:
            beta, value = next_beta, next_value
    return beta

In [17]:
steps = [100, 10, 1, 0.1, 0.01, 0.001, 0.0001, 0.00001]
gradient_descent_num(target_fn_num,grad_fn_num,steps)

array([ -2.84953788,  43.20412672])

Теперь конечно же проверим, что даст нам лёрновская линейная регрессия.

In [18]:
from sklearn.linear_model import LinearRegression

In [19]:
clf = LinearRegression()

In [20]:
clf.fit(x[:,1:],y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [21]:
clf.coef_,clf.intercept_

(array([ 43.20424388]), -2.8496363946075389)

Значения очень похожи, а разняться наверное потому, что sklearn использует какой-то другой алгоритм для решения задачи регрессии.

Градиентный спуск - очень понятный метод оптимизации целевой функции, который постоянно используется в решении задач машинного обучения. Его применяют при решении задач линейной регрессии, логистической регрессии, нейронных сетях и др. В данной статье была рассмотрена пакетная (batch) версия алгоритма градиентного спуска, который для расчета шага использует все данные **сразу**. Более популярный алгоритм - это стохастический градиентный спуск. Он считает шаг для каждого элемента выборки, которые берутся в случайном (стохастическом) порядке. Этот алгоритм позволяет решать задачи оптимизации на очень больших массивах данным, которые не могут храниться в оперативной памяти все сразу.

Данная статья написанапо материалам совершенно замечальной книги по машинному обучению. Она называется [Data Science From Scratch](http://shop.oreilly.com/product/0636920033400.do) и написал ее Joel Grus. Очень рекомендую эту книгу, а так же [github автора](https://github.com/joelgrus/data-science-from-scratch).